In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import importlib

import scripts.preprocessing as preprocessing
importlib.reload(preprocessing)

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torchsummary
import torch.optim as optim
import scripts.models as models
import tqdm
import mlflow
import mlflow.pytorch

from scripts.utils import *

importlib.reload(models)

<module 'scripts.models' from '/home/claudio/Scrivania/Supervised Learning/food_SSL/scripts/models.py'>

In [2]:
#set the correct directory for mlflow tracking
#mlflow.set_tracking_uri("file:./mlruns")

In [3]:
device = use_GPU()

CUDA is not available.  Training on CPU ...
cpu


/home/claudio/anaconda3/envs/pytorch/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
model = models.SqueezeNet()
model = model.to(device)

In [5]:
torchsummary.summary(model, (3, 224, 224));

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 111, 111]           1,792
              ReLU-2         [-1, 64, 111, 111]               0
         MaxPool2d-3           [-1, 64, 55, 55]               0
            Conv2d-4           [-1, 16, 55, 55]           1,040
              ReLU-5           [-1, 16, 55, 55]               0
            Conv2d-6           [-1, 64, 55, 55]           1,088
              ReLU-7           [-1, 64, 55, 55]               0
            Conv2d-8           [-1, 64, 55, 55]           9,280
              ReLU-9           [-1, 64, 55, 55]               0
             Fire-10          [-1, 128, 55, 55]               0
           Conv2d-11           [-1, 16, 55, 55]           2,064
             ReLU-12           [-1, 16, 55, 55]               0
           Conv2d-13           [-1, 64, 55, 55]           1,088
             ReLU-14           [-1, 64,

In [6]:
# Load data
folder_structure = preprocessing.create_dataset()
# transform it in a dataframe and list the number of images per class in the folders
a = pd.DataFrame([(k, len(v)) for k,v in folder_structure[0].items()], 
                    columns=['class', 'count'])
b = pd.DataFrame([(k, len(v)) for k,v in folder_structure[1].items()], 
                    columns=['class', 'count'])
image_counts = pd.merge(a, 
                        b, 
                        on='class', 
                        how='outer', 
                        suffixes=('_train', '_test'))

Populating data/train
Populating data/test


In [7]:
image_counts.head()

,class,count_train,count_test
0,adobo,498,43
1,ambrosia_food,569,35
2,apple_pie,549,41
3,apple_turnover,504,43
4,applesauce,484,39


In [8]:
image_counts.loc[np.argmin(image_counts['count_train']),:]

class          marble_cake
count_train             34
count_test              49
Name: 143, dtype: object

In [9]:
preprocessing.create_validation(42);

In [10]:
transform = transforms.Compose([
    # resize 
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    # Normalize pixel values
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the training dataset
trainset = torchvision.datasets.ImageFolder(root='data/train', transform=transform)

# Create data loader for training data with batch size 4 and shuffling
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

valset = torchvision.datasets.ImageFolder(root='data/val', transform=transform)

valloader = torch.utils.data.DataLoader(valset, batch_size=16, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='data/test', transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)

In [11]:
# Define criterion, optimizer and scheduler and other parameters for training
opt = "Adam"                        # optimizer to be used: ["Adam" or "SGD"]
momentum = 0.9                      # momentum ONLY for SGD optimizer
weight_decay = 1e-4                 # weight decay ONLY on Adam optimizer
step_size = 7                       # step size for the scheduler
gamma = 0.1                         # gamma for the scheduler

batch_size = 8                      # batch size
num_epochs=10                       # number of epochs
patience = 3                        # patience for early stopping
criterion ="CrossEntropyLoss"       # loss function to be used: ["CrossEntropyLoss", "MSELoss", "L1Loss", "NLLLoss"]
lr = 5e-5                           # learning rate

model_name = "squeezenet"           # model name
model = models.SqueezeNet()         # model



#set the optimizer
if opt == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
elif opt == "SGD":
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
else:
    print("Invalid optimizer")

#set the criterion
if criterion == "CrossEntropyLoss":
    criterion = nn.CrossEntropyLoss()
elif criterion == "MSELoss":
    criterion = nn.MSELoss()
elif criterion == "L1Loss":
    criterion = nn.L1Loss()
elif criterion == "NLLLoss":
    criterion = nn.NLLLoss()
else:
    print("Invalid criterion")

#set the scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# Upload model to correct device
model = model.to(device)

In [17]:
def set_training_parameters(model, model_name, opt, lr, weight_decay, momentum, criterion, step_size, gamma, num_epochs, patience, device):

    #set the optimizer
    if opt == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif opt == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    else:
        print("Invalid optimizer")

    #set the criterion
    if criterion == "CrossEntropyLoss":
        criterion = nn.CrossEntropyLoss()
    elif criterion == "MSELoss":
        criterion = nn.MSELoss()
    elif criterion == "L1Loss":
        criterion = nn.L1Loss()
    elif criterion == "NLLLoss":
        criterion = nn.NLLLoss()
    else:
        print("Invalid criterion")

    #set the scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Upload model to correct device
    model = model.to(device)
    
    training_parameters = {}
    training_parameters['model'] = model
    training_parameters['model_name'] = model_name
    training_parameters['criterion'] = criterion
    training_parameters['optimizer'] = optimizer
    training_parameters['scheduler'] = scheduler
    training_parameters['num_epochs'] = num_epochs
    training_parameters['patience'] = patience
    
    return training_parameters

In [19]:
# Training function based on above parameters
def train_model(model, model_name, trainloader, valloader, criterion, optimizer, scheduler, num_epochs=10, patience=3 ):

    mlflow.start_run(run_name=model_name)

    # Log model parameters
    mlflow.log_param("optimizer", opt)
    mlflow.log_param("learning_rate", lr)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("momentum", momentum)
    mlflow.log_param("weight_decay", weight_decay)
    mlflow.log_param("step_size", step_size)
    mlflow.log_param("gamma", gamma)
    mlflow.log_param("patience", patience)


    patience_counter = 0
    best_model = None
    best_loss = np.inf

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        train_loader_tqdm = tqdm.tqdm(trainloader, desc=f"Epoch {epoch+1}/{num_epochs}", 
                                unit="batch")
        
        for inputs, labels in train_loader_tqdm:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            train_loader_tqdm.set_postfix(loss=running_loss / len(trainloader))

        epoch_loss = running_loss / len(trainloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

        scheduler.step()
        mlflow.log_metric("train_loss", epoch_loss, step=epoch)
        
        # Validation loop (optional)
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(valloader)
        val_accuracy = 100 * correct / total
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
        
        # Log validation loss and accuracy
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("val_accuracy", val_accuracy, step=epoch)
        
        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter > patience:
                print("Early stopping")
                break
                    
    # Log the model
    mlflow.pytorch.log_model(best_model, model_name)

    # End the MLflow run
    mlflow.end_run()

    print('Finished Training')


In [20]:
# example of training 

# Define criterion, optimizer and scheduler and other parameters for training
opt = "Adam"                        # optimizer to be used: ["Adam" or "SGD"]
momentum = 0.9                      # momentum ONLY for SGD optimizer
weight_decay = 1e-4                 # weight decay ONLY on Adam optimizer
step_size = 7                       # step size for the scheduler
gamma = 1                         # gamma for the scheduler

batch_size = 32                     # batch size
num_epochs= 30                        # number of epochs
patience = 3                        # patience for early stopping
criterion ="CrossEntropyLoss"       # loss function to be used: ["CrossEntropyLoss", "MSELoss", "L1Loss", "NLLLoss"]
lr = 0.001                        # learning rate

model_name = "squeezenet"           # model name
model = models.SqueezeNet()         # model

tr_param = set_training_parameters(model=model,model_name = model_name, opt=opt, lr=lr, weight_decay=weight_decay, 
                                   momentum=momentum, criterion=criterion, step_size=step_size, gamma=gamma, num_epochs=num_epochs, 
                                   patience=patience, device=device)

#stop eventual mlflow runs
mlflow.end_run()
train_model(**tr_param, trainloader=trainloader, valloader=valloader)

Epoch 1/30:  12%|█▏        | 1473/11860 [12:01<1:24:49,  2.04batch/s, loss=0.686]


KeyboardInterrupt: 

In [ ]:
model_from_scratch = models.SqueezeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

mlflow.end_run()
train_model(model_from_scratch,
            "netFromScratch", 
            trainloader, 
            valloader, 
            criterion, 
            optimizer, 
            scheduler,
            num_epochs=10, 
            device=device)

Epoch 1/10: 100%|██████████| 14810/14810 [21:25<00:00, 11.52batch/s, loss=5.6] 

Epoch [1/10], Loss: 5.6000


Validation Loss: 5.6158, Validation Accuracy: 0.41%


Epoch 2/10: 100%|██████████| 14810/14810 [21:59<00:00, 11.23batch/s, loss=5.6] 

Epoch [2/10], Loss: 5.6001


Validation Loss: 5.6157, Validation Accuracy: 0.40%


Epoch 3/10:  39%|███▉      | 5845/14810 [09:11<14:05, 10.60batch/s, loss=2.21]   


KeyboardInterrupt: 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#mlflow.end_run()
model = models.SqueezeNet()
model.load_state_dict(torch.load('netFromSSL.pth'))
train_model(model,
            "netFromSSL",
            trainloader,
            valloader,
            criterion,
            optimizer,
            scheduler,
            num_epochs=4,
            device=device)

Epoch 1/4: 100%|██████████| 14810/14810 [22:05<00:00, 11.18batch/s, loss=5.55]  

Epoch [1/4], Loss: 5.5479


Validation Loss: 5.5522, Validation Accuracy: 0.39%


Epoch 2/4: 100%|██████████| 14810/14810 [21:32<00:00, 11.46batch/s, loss=5.55]

Epoch [2/4], Loss: 5.5481


...

Epoch [3/4], Loss: 5.5479


Validation Loss: 5.5522, Validation Accuracy: 0.39%


Epoch 4/4: 100%|██████████| 14810/14810 [21:28<00:00, 11.49batch/s, loss=5.55]

Epoch [4/4], Loss: 5.5478


Validation Loss: 5.5522, Validation Accuracy: 0.39%
Finished Training


In [ ]:
mlflow.end_run()

In [ ]:
#best hyperparameters from github non provato

# transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
}

data_dir = 'path_to_foodx251_dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = models.squeezenet1_1(pretrained=True)
model.classifier[1] = nn.Conv2d(512, len(class_names), kernel_size=(1,1), stride=(1,1))
model.num_classes = len(class_names)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / dataset_sizes['train']
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')
        scheduler.step()

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss /= dataset_sizes['val']
        val_accuracy = correct / total
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

    return model

# Train the model
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25)